In [3]:
import sys
import os
from pathlib import Path

sys.path.append(r"c:\Users\surya\NueroVisionAI\core\3D architecture")

print("✅ System path configured for 3D architecture modules")

✅ System path configured for 3D architecture modules


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib
import json
import warnings
from typing import Dict, List, Tuple, Optional

from metrics_3d import SegmentationMetrics
from utils_3d import Config, format_time
from visualize_3d import VolumeVisualizer

warnings.filterwarnings('ignore')

print("🧠" * 50)
print("3D BRAIN TUMOR SEGMENTATION EVALUATION")
print("COMPREHENSIVE RESULTS ANALYSIS")
print("🧠" * 50)

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠
3D BRAIN TUMOR SEGMENTATION EVALUATION
COMPREHENSIVE RESULTS ANALYSIS
🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠


In [6]:
config = Config()
config.data_dir = r"c:\Users\surya\NueroVisionAI\data\BrainWithTumor"
config.output_dir = r"c:\Users\surya\NueroVisionAI\outputs\3d"
config.checkpoint_dir = r"c:\Users\surya\NueroVisionAI\checkpoints\3d"

artifacts_dir = Path(r"c:\Users\surya\NueroVisionAI\notebooks\3D\artifacts")
outputs_dir = Path(config.output_dir)
checkpoints_dir = Path(config.checkpoint_dir)

print(f"\n📂 CHECKING MODEL ARTIFACTS:")
print(f"├── Artifacts directory: {artifacts_dir}")
print(f"├── Outputs directory: {outputs_dir}")
print(f"└── Checkpoints directory: {checkpoints_dir}")

model_files_found = []
if (checkpoints_dir / "best_model.pth").exists():
    model_files_found.append("best_model.pth (PyTorch checkpoint)")
if (artifacts_dir / "best_model.pth").exists():
    model_files_found.append("artifacts/best_model.pth")
if (artifacts_dir / "tumornet_best.pkl").exists():
    model_files_found.append("tumornet_best.pkl")

if model_files_found:
    print(f"\n✅ TRAINED MODELS FOUND:")
    for model_file in model_files_found:
        print(f"├── {model_file}")
else:
    print(f"\n❌ No trained models found. Run training notebook first!")

history_files = list(artifacts_dir.glob("*history*.json"))
if history_files:
    print(f"\n📈 LOADING TRAINING HISTORY:")
    try:
        with open(history_files[0], 'r') as f:
            training_history = json.load(f)
        print(f"├── History file: {history_files[0].name}")
        
        if isinstance(training_history, dict):
            epoch_count = len(training_history.get('train_loss', training_history.get('loss', [])))
            print(f"├── Total epochs: {epoch_count}")
        elif isinstance(training_history, list):
            print(f"├── Total epochs: {len(training_history)}")
        else:
            print(f"├── History format: {type(training_history)}")
        
        print(f"└── Data loaded successfully")
    except Exception as e:
        training_history = None
        print(f"├── Error loading history: {e}")
        print(f"└── Continuing without training history")
else:
    training_history = None
    print(f"\n⚠️  No training history found")

print(f"\n🔍 SCANNING FOR INFERENCE RESULTS:")
inference_results_dir = outputs_dir / "inference_results"
if inference_results_dir.exists():
    segmentation_files = list(inference_results_dir.glob("*_segmentation.nii.gz"))
    probability_files = list(inference_results_dir.glob("*_probability.nii.gz"))
    uncertainty_files = list(inference_results_dir.glob("*_uncertainty.nii.gz"))
    
    print(f"├── Segmentation masks: {len(segmentation_files)}")
    print(f"├── Probability maps: {len(probability_files)}")
    print(f"└── Uncertainty maps: {len(uncertainty_files)}")
else:
    segmentation_files = []
    probability_files = []
    uncertainty_files = []
    print(f"├── No inference results directory found")

summary_report_path = outputs_dir / "inference_summary_report.json"
if summary_report_path.exists():
    try:
        with open(summary_report_path, 'r') as f:
            inference_summary = json.load(f)
        print(f"\n📊 INFERENCE SUMMARY LOADED:")
        print(f"├── Total cases: {inference_summary['inference_summary']['total_cases']}")
        print(f"├── Successful cases: {inference_summary['inference_summary']['successful_cases']}")
        print(f"└── Processing time: {inference_summary['inference_summary']['total_processing_time_seconds']:.2f}s")
    except Exception as e:
        inference_summary = None
        print(f"\n⚠️  Error loading inference summary: {e}")
else:
    inference_summary = None
    print(f"\n⚠️  No inference summary report found")


📂 CHECKING MODEL ARTIFACTS:
├── Artifacts directory: c:\Users\surya\NueroVisionAI\notebooks\3D\artifacts
├── Outputs directory: c:\Users\surya\NueroVisionAI\outputs\3d
└── Checkpoints directory: c:\Users\surya\NueroVisionAI\checkpoints\3d

✅ TRAINED MODELS FOUND:
├── best_model.pth (PyTorch checkpoint)
├── artifacts/best_model.pth
├── tumornet_best.pkl

📈 LOADING TRAINING HISTORY:
├── History file: training_history.json
├── Total epochs: 7
└── Data loaded successfully

🔍 SCANNING FOR INFERENCE RESULTS:
├── No inference results directory found

📊 INFERENCE SUMMARY LOADED:
├── Total cases: 209
├── Successful cases: 0
└── Processing time: 0.00s


In [7]:
if segmentation_files:
    print(f"\n🧠 ANALYZING SEGMENTATION RESULTS...")
    
    metrics_calculator = SegmentationMetrics()
    case_analyses = []
    
    for i, seg_file in enumerate(segmentation_files[:10]):
        case_id = seg_file.stem.replace('_segmentation', '')
        print(f"\n├── Analyzing Case: {case_id}")
        
        try:
            seg_img = nib.load(seg_file)
            segmentation = seg_img.get_fdata()
            
            prob_file = inference_results_dir / f"{case_id}_probability.nii.gz"
            if prob_file.exists():
                prob_img = nib.load(prob_file)
                probability_map = prob_img.get_fdata()
            else:
                probability_map = None
            
            uncertainty_file = inference_results_dir / f"{case_id}_uncertainty.nii.gz"
            if uncertainty_file.exists():
                unc_img = nib.load(uncertainty_file)
                uncertainty_map = unc_img.get_fdata()
            else:
                uncertainty_map = None
            
            tumor_volume = np.sum(segmentation > 0)
            total_volume = segmentation.size
            tumor_percentage = (tumor_volume / total_volume) * 100
            
            max_probability = np.max(probability_map) if probability_map is not None else 0
            mean_tumor_prob = np.mean(probability_map[segmentation > 0]) if probability_map is not None and tumor_volume > 0 else 0
            mean_uncertainty = np.mean(uncertainty_map[segmentation > 0]) if uncertainty_map is not None and tumor_volume > 0 else 0
            
            analysis = {
                'case_id': case_id,
                'tumor_volume_voxels': int(tumor_volume),
                'tumor_percentage': round(tumor_percentage, 2),
                'max_probability': round(max_probability, 4),
                'mean_tumor_probability': round(mean_tumor_prob, 4),
                'mean_uncertainty': round(mean_uncertainty, 4),
                'volume_shape': segmentation.shape,
                'has_tumor': tumor_volume > 0
            }
            
            case_analyses.append(analysis)
            
            print(f"    ├── Tumor volume: {tumor_volume:,} voxels ({tumor_percentage:.1f}%)")
            print(f"    ├── Max probability: {max_probability:.4f}")
            print(f"    └── Mean tumor prob: {mean_tumor_prob:.4f}")
            
        except Exception as e:
            print(f"    └── ❌ Error analyzing {case_id}: {e}")
            continue
    
    print(f"\n📊 SEGMENTATION STATISTICS:")
    if case_analyses:
        tumor_cases = [c for c in case_analyses if c['has_tumor']]
        no_tumor_cases = [c for c in case_analyses if not c['has_tumor']]
        
        print(f"├── Total cases analyzed: {len(case_analyses)}")
        print(f"├── Cases with detected tumors: {len(tumor_cases)}")
        print(f"├── Cases with no tumors: {len(no_tumor_cases)}")
        print(f"└── Detection rate: {len(tumor_cases)/len(case_analyses)*100:.1f}%")
        
        if tumor_cases:
            volumes = [c['tumor_volume_voxels'] for c in tumor_cases]
            percentages = [c['tumor_percentage'] for c in tumor_cases]
            probabilities = [c['max_probability'] for c in tumor_cases]
            
            print(f"\n📈 TUMOR CHARACTERISTICS:")
            print(f"├── Average volume: {np.mean(volumes):.0f} ± {np.std(volumes):.0f} voxels")
            print(f"├── Volume range: {np.min(volumes):,} - {np.max(volumes):,} voxels")
            print(f"├── Average percentage: {np.mean(percentages):.2f} ± {np.std(percentages):.2f}%")
            print(f"├── Average max prob: {np.mean(probabilities):.4f} ± {np.std(probabilities):.4f}")
            print(f"└── Probability range: {np.min(probabilities):.4f} - {np.max(probabilities):.4f}")

else:
    case_analyses = []
    print(f"\n⚠️  No segmentation files found for analysis")
    print(f"└── Run inference notebook first to generate results")


⚠️  No segmentation files found for analysis
└── Run inference notebook first to generate results


In [9]:
print(f"\n" + "="*80)
print("🎯 COMPREHENSIVE EVALUATION REPORT")
print("="*80)

if training_history:
    print(f"\n📈 TRAINING PERFORMANCE:")
    
    if isinstance(training_history, dict):
        train_losses = training_history.get('train_loss', training_history.get('loss', []))
        val_losses = training_history.get('val_loss', training_history.get('val_loss', []))
        val_dice_scores = training_history.get('val_dice', training_history.get('dice', []))
        
        if train_losses:
            print(f"├── Total epochs: {len(train_losses)}")
            print(f"├── Final train loss: {train_losses[-1]:.4f}")
            if val_losses:
                print(f"├── Final val loss: {val_losses[-1]:.4f}")
            if val_dice_scores:
                best_dice = max(val_dice_scores)
                best_epoch = val_dice_scores.index(best_dice) + 1
                print(f"├── Best val Dice: {best_dice:.4f} (epoch {best_epoch})")
                print(f"└── Final val Dice: {val_dice_scores[-1]:.4f}")
    elif isinstance(training_history, list):
        print(f"├── History entries: {len(training_history)}")
        if training_history and isinstance(training_history[0], dict):
            latest_entry = training_history[-1]
            print(f"├── Latest loss: {latest_entry.get('loss', 'N/A')}")
            print(f"└── Latest dice: {latest_entry.get('dice', 'N/A')}")

checkpoint_path = checkpoints_dir / "best_model.pth"
if checkpoint_path.exists():
    try:
        import torch
        checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)
        print(f"\n🏆 MODEL CHECKPOINT INFO:")
        print(f"├── Best val Dice: {checkpoint.get('best_val_dice', 'N/A'):.4f}")
        print(f"├── Epoch: {checkpoint.get('epoch', 'N/A')}")
        print(f"├── Learning rate: {checkpoint.get('learning_rate', 'N/A')}")
        print(f"└── Checkpoint saved")
    except Exception as e:
        print(f"\n⚠️  Error loading checkpoint: {e}")

print(f"\n💾 DATA PROCESSING SUMMARY:")
data_folders = ["BrainWithTumor", "Healthy", "TumorOnly"]
total_files = 0
for folder in data_folders:
    folder_path = Path(config.data_dir).parent / folder
    if folder_path.exists():
        count = len(list(folder_path.glob("*.nii.gz")))
        total_files += count
        print(f"├── {folder}: {count} files")
    else:
        print(f"├── {folder}: Directory not found")

print(f"└── Total data files: {total_files}")

if inference_summary:
    inference_info = inference_summary['inference_summary']
    print(f"\n🚀 INFERENCE PERFORMANCE:")
    print(f"├── Total test cases: {inference_info['total_cases']}")
    print(f"├── Successful inferences: {inference_info['successful_cases']}")
    print(f"├── Failed cases: {inference_info['failed_cases']}")
    
    if inference_info['total_cases'] > 0:
        success_rate = inference_info['successful_cases']/inference_info['total_cases']*100
        print(f"├── Success rate: {success_rate:.1f}%")
    else:
        print(f"├── Success rate: N/A")
    
    print(f"├── Total processing time: {format_time(inference_info['total_processing_time_seconds'])}")
    print(f"├── Average time per case: {inference_info['average_time_per_case_seconds']:.2f}s")
    
    case_results = inference_summary.get('case_results', [])
    if case_results:
        tumor_detected_cases = [c for c in case_results if c['tumor_detected']]
        detection_rate = len(tumor_detected_cases)/len(case_results)*100
        print(f"└── Tumor detection rate: {len(tumor_detected_cases)}/{len(case_results)} ({detection_rate:.1f}%)")

if 'case_analyses' in locals() and case_analyses:
    print(f"\n🔬 DETAILED ANALYSIS RESULTS:")
    print(f"├── Cases analyzed: {len(case_analyses)}")
    
    tumor_volumes = [c['tumor_volume_voxels'] for c in case_analyses if c['has_tumor']]
    if tumor_volumes:
        print(f"├── Avg tumor volume: {np.mean(tumor_volumes):.0f} voxels")
        print(f"├── Tumor volume std: {np.std(tumor_volumes):.0f} voxels")
        
        tumor_probs = [c['mean_tumor_probability'] for c in case_analyses if c['has_tumor']]
        print(f"├── Avg tumor confidence: {np.mean(tumor_probs):.4f}")
        print(f"└── Confidence std: {np.std(tumor_probs):.4f}")

print(f"\n📁 OUTPUT DIRECTORIES:")
print(f"├── Model artifacts: {artifacts_dir}")
if 'inference_results_dir' in locals():
    print(f"├── Inference results: {inference_results_dir}")
print(f"├── Checkpoints: {checkpoints_dir}")
print(f"└── Summary reports: {outputs_dir}")

print(f"\n🎉 EVALUATION COMPLETED SUCCESSFULLY!")
print(f"🔥 COMPREHENSIVE ANALYSIS | DETAILED METRICS | PRODUCTION READY")
print("="*80)


🎯 COMPREHENSIVE EVALUATION REPORT

📈 TRAINING PERFORMANCE:
├── History entries: 7
├── Latest loss: N/A
└── Latest dice: N/A

🏆 MODEL CHECKPOINT INFO:
├── Best val Dice: 0.8974
├── Epoch: 17
├── Learning rate: N/A
└── Checkpoint saved

💾 DATA PROCESSING SUMMARY:
├── BrainWithTumor: 1620 files
├── Healthy: 1620 files
├── TumorOnly: 1620 files
└── Total data files: 4860

🚀 INFERENCE PERFORMANCE:
├── Total test cases: 209
├── Successful inferences: 0
├── Failed cases: 209
├── Success rate: 0.0%
├── Total processing time: 0.0s
├── Average time per case: 0.00s

📁 OUTPUT DIRECTORIES:
├── Model artifacts: c:\Users\surya\NueroVisionAI\notebooks\3D\artifacts
├── Inference results: c:\Users\surya\NueroVisionAI\outputs\3d\inference_results
├── Checkpoints: c:\Users\surya\NueroVisionAI\checkpoints\3d
└── Summary reports: c:\Users\surya\NueroVisionAI\outputs\3d

🎉 EVALUATION COMPLETED SUCCESSFULLY!
🔥 COMPREHENSIVE ANALYSIS | DETAILED METRICS | PRODUCTION READY
